In [1]:
# Import the dependencies.
# Import the datetime module from the datetime library.
from datetime import datetime
import pandas as pd
import numpy as np
import requests
from citipy import citipy
import json
import subprocess
import psycopg2

# Import the API key.
from config import client_id, client_secret, redirect_url, base_url, oauth_url

url = "https://api.procore.com/oauth/token?grant_type=client_credentials&client_id=" + client_id + "&client_secret=" + client_secret

In [2]:
## Load Data from Project Database T:\zz Tools\Project Database.xlsx

projects_df = pd.read_excel(open('T:/zz Tools/Project Database.xlsm', 'rb'), sheet_name='EVERYTHING') 

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [3]:
## Set first column as index
projects_df = projects_df.set_index("Unnamed: 0")

## Rename col 1 to correct description
projects_df = projects_df.rename(columns={"Unnamed: 1": "Description"})

projects_df

,Description,Job 1,Job 2,Job 3,Job 4,Job 5,Job 6,Job 7,Job 8,Job 9,...,Job 500,-,Unnamed: 503,Unnamed: 504,Unnamed: 505,Unnamed: 506,Unnamed: 507,Unnamed: 508,Unnamed: 509,Unnamed: 510
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Project #,Show highlights,15002,15003,15004,15005,15006,16001,16002,16003,16004,...,NaN,-,NaN,Composite Averages,NaN,NaN,NaN,NaN,NaN,NaN
Developer,no,OHT Partners,OHT Partners,OHT Partners,Ardent,IMT,Ardent,GenCap,OHT Partners,Centro,...,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Project Name,Required?,Kingsland,Lenox Boardwalk - Pleasant Valley,Agora Palms,Ocotillo,Riata,Farmhouse,Rialto,Lenox Springs,Tarlton 360,...,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Last Updated,NaN,2017-10-24 00:00:00,2018-04-20 00:00:00,2017-05-10 00:00:00,NaN,NaN,2017-10-05 00:00:00,NaN,2018-06-20 00:00:00,NaN,...,NaN,-,Date Filter,not set up,NaN,NaN,NaN,NaN,NaN,NaN
Project Type,NaN,Garden,Wrap,Garden,Garden,Garden,Garden,Garden,Garden,Podium,...,NaN,-,Select Type,Garden,Wrap,Podium,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
06-01-250,Amenity Appliances,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7957.868814,8463.263143,5472.1312,NaN,NaN,NaN,NaN
50-01-100,CM Contingency,136041.84,0,NaN,NaN,NaN,715588,NaN,189355,NaN,...,NaN,NaN,NaN,877375.571381,1085959.215583,963899.822458,NaN,NaN,NaN,NaN
50-01-102,Escalation Contingency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
## Select Test Project

push_test = projects_df["Job 50"]
push_test

push = push_test[0:33]
push

Unnamed: 0
Project #                                18003
Developer                                  CWS
Project Name                View on the Square
Last Updated               2018-04-03 00:00:00
Project Type                            Podium
Project Sub-Type      5-Story - Precast Garage
Special Type                           Student
Region                              San Marcos
Project Manager                            NaN
Fixture/Finish Lvl                         NaN
Job Status                              Canned
Soil Conditions                           rock
WQ Pond                                    NaN
Det. Pond                                  NaN
PC Garage Spaces                           NaN
CIP Garage Spaces                          NaN
Levels UG Parking                          NaN
Acres                                        1
Buildings                                    1
Framed Stories                             NaN
Apartments                                 120
To

In [9]:
push.rename({"Project #": "project_number", "Project Name": "name", "Last Updated": "updated_at", "Project Type": "project_type", "Project Sub-Type": "custom_field_235145", "Special Type": "departments", "Region": "city", "Soil Conditions": "custom_field_234689", "Acres": "custom_field_234690", "Buildings": "custom_field_234691", "Apartments": "custom_field_234819", "Bedrooms": "custom_field_234692"})

Unnamed: 0
project_number                            18003
Developer                                   CWS
name                         View on the Square
updated_at                  2018-04-03 00:00:00
project_type                             Podium
custom_field_235145    5-Story - Precast Garage
departments                             Student
city                                 San Marcos
Project Manager                             NaN
Fixture/Finish Lvl                          NaN
Job Status                               Canned
custom_field_234689                        rock
WQ Pond                                     NaN
Det. Pond                                   NaN
PC Garage Spaces                            NaN
CIP Garage Spaces                           NaN
Levels UG Parking                           NaN
custom_field_234690                           1
custom_field_234691                           1
Framed Stories                              NaN
custom_field_234819          